In [1]:
import torch 
import torch.nn as nn
import numpy as np

In [2]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token, end_token):

        def tokenize(sentence, start_token, end_token):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())
    
    def forward(self, x, start_token, end_token): # sentence
        x = self.batch_tokenize(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

  
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x


class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x.clone()
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x
    
class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

class Encoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask)
        return x


class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # in practice, this is the same for both languages...so we can technically combine with normal attention
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask) # We don't need the mask for cross attention, removing in outer function!
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out


class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y


class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

class Decoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y


class Transformer(nn.Module):
    def __init__(self, 
                d_model, 
                ffn_hidden, 
                num_heads, 
                drop_prob, 
                num_layers,
                max_sequence_length, 
                kn_vocab_size,
                english_to_index,
                kannada_to_index,
                START_TOKEN, 
                END_TOKEN, 
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, kannada_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, kn_vocab_size)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def forward(self, 
                x, 
                y, 
                encoder_self_attention_mask=None, 
                decoder_self_attention_mask=None, 
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out

In [3]:
english_file = 'english.txt'
kannada_file = 'kannada.txt'


In [4]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

kannada_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ', 
                      'ँ', 'ఆ', 'ఇ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 
                      'ಅ', 'ಆ', 'ಇ', 'ಈ', 'ಉ', 'ಊ', 'ಋ', 'ೠ', 'ಌ', 'ಎ', 'ಏ', 'ಐ', 'ಒ', 'ಓ', 'ಔ', 
                      'ಕ', 'ಖ', 'ಗ', 'ಘ', 'ಙ', 
                      'ಚ', 'ಛ', 'ಜ', 'ಝ', 'ಞ', 
                      'ಟ', 'ಠ', 'ಡ', 'ಢ', 'ಣ', 
                      'ತ', 'ಥ', 'ದ', 'ಧ', 'ನ', 
                      'ಪ', 'ಫ', 'ಬ', 'ಭ', 'ಮ', 
                      'ಯ', 'ರ', 'ಱ', 'ಲ', 'ಳ', 'ವ', 'ಶ', 'ಷ', 'ಸ', 'ಹ', 
                      '಼', 'ಽ', 'ಾ', 'ಿ', 'ೀ', 'ು', 'ೂ', 'ೃ', 'ೄ', 'ೆ', 'ೇ', 'ೈ', 'ೊ', 'ೋ', 'ೌ', '್', 'ೕ', 'ೖ', 'ೞ', 'ೣ', 'ಂ', 'ಃ', 
                      '೦', '೧', '೨', '೩', '೪', '೫', '೬', '೭', '೮', '೯', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@', 
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                        'Y', 'Z',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [5]:
index_to_kannada = {k:v for k,v in enumerate(kannada_vocabulary)}
kannada_to_index = {v:k for k,v in enumerate(kannada_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [6]:
with open(english_file, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()

with open(kannada_file, 'r', encoding='utf-8') as file:
    kannada_sentences = file.readlines()

In [7]:
#limit number of sentences

In [8]:
# Limit Number of sentences
TOTAL_SENTENCES = 100000
english_sentences = english_sentences[:TOTAL_SENTENCES]
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
kannada_sentences = [sentence.rstrip('\n') for sentence in kannada_sentences]

In [9]:
# finding the max lenght of sentences in each language
a = []
for i in english_sentences:
    a.append(len(i))
print(max(a))    
b = []
for i in kannada_sentences:
    b.append(len(i))
print(max(b))

722
639


In [10]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1)   # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = kannada_sentences[index], english_sentences[index]
    if is_valid_length(kannada_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(kannada_sentence, kannada_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 100000
Number of valid sentences: 81916


In [11]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [12]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(kannada_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          kannada_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [13]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(97, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [14]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [15]:
dataset = TextDataset(english_sentences, kannada_sentences)

In [16]:
dataset[1]

("'But we speak the truth aur ye sach hai ke Gujarat mein vikas pagal hogaya hai,'' Rahul Gandhi further said in Banaskantha",
 '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"')

In [17]:
batch_size = 3 
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [18]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('Hes a scientist.', "'But we speak the truth aur ye sach hai ke Gujarat mein vikas pagal hogaya hai,'' Rahul Gandhi further said in Banaskantha", '8 lakh crore have been looted.'), ('ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.', '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"', 'ಕಳ್ಳತನವಾಗಿದ್ದ 8 ಲಕ್ಷ ರೂ.')]
[('I read a lot into this as well.', 'How did mankind come under Satans rival sovereignty?', 'And then I became Prime Minister.'), ('ಇದರ ಬಗ್ಗೆ ನಾನೂ ಸಾಕಷ್ಟು ಓದಿದ್ದೇನೆ.', 'ಮಾನವಕುಲವು ಸೈತಾನನ ಆಳಿಕೆಯ ಕೆಳಗೆ ಬಂದದ್ದು ಹೇಗೆ?', 'ನಂತರ ಪ್ರಧಾನಿ ಕೂಡ ಆಗುತ್ತೇನೆ.')]
[('What about corruption?', '"""The shooting of the film is 90 percent done."', 'the Special Statute'), ('ಭ್ರಷ್ಟಾಚಾರ ಏಕಿದೆ?', 'ಆ ಚಿತ್ರದ ಶೇ 90ರಷ್ಟು ಚಿತ್ರೀಕರಣವೂ ಈಗಾಗಲೇ ಮುಗಿದು ಹೋಗಿದೆ.', 'ವಿಶೇಷ ಕಾನೂನು')]
[('"Then the king said to Ittai the Gittite, ""Why do you also go with us? Return, and stay with the king. for you are a foreigner, and also an exile. Return to your own place."', 'What happened at the UN Ge

In [19]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=kannada_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [20]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
        eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
        kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Kannada Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
                if idx == kannada_to_index[END_TOKEN]:
                    break
                predicted_sentence += index_to_kannada[idx.item()]
            print(f"Kannada Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break
            
            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.32479190826416
English: Hes a scientist.
Kannada Translation: ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.
Kannada Prediction: ?ಯಹಯಲಹ=ಱಱ(ಱೈ+ಯೈಫಲ%ೌ=ೈಱಱಱೈ೯ಹ೯++ೈಡೈಧ+ನೊೊೈೊನೊನಫ6ತ6೫<6ಷತಯಯ೫ಯ<<ಢಷಯಯಙూಯಯಯ<7ˌ77ತ%7ಯಷిಯಯಯಷ6ತಹ೨ೊತ17ತಯಯಹ=ಷದಷಣಹ೭7ಯಯಯತಯ=ೊಯಫ<ಫ4ಹಏತೊೊತಢ77ಢಯ೫ೊೊೊೊ3ೊಯ
Evaluation translation (should we go to the mall?) : ('್್್ಲಲಲಾಾಸದ+ದರದದದಾವ+++್್್ಾಾಾನನನನ+ನನನನಾಾಾಾಾಾಾಾಾಾಾಾಾಾಾಾಯಯಯಯಯಯಯಯಯಯಯಯಯಯಯದದದಯಯಯಯిిిಯಯಯಯಯಯಯಯಯಯ೭೭೭ಯಯದದದ೭೭ಙ್್್್೭೭೭೭೭೭೭೭೭  ೭೭೭     ್   ೫೫ಏೊ೭  <END>',)
-------------------------------------------
Iteration 100 : 3.494189977645874
English: I am not saying this.
Kannada Translation: ಈ ಮಾತನ್ನು ನಾನಲ್ಲ.
Kannada Prediction: ುನು್ು ್    ್ದಾುು್ುಿಿಿ್ ್್್್ರಿ್ ್ ್ದದಿ ಿಿದ್್್ರ ದದ ಿದದ್್್ರರಿ ದಿು್ಿ್ಿಿಿಿದುಿ ಿಿರ ್ಿಿ ್ುಿಿೆರ್ುು್ದಗದೆೆಿ್್ಿಿಿಿ್್್್್್ುುದಿುುುರುೆ್ೆಿೆ ೆಿಿದ್ಿ್ೆುಿಿೆೆರೆುುಿಿಿ್ೆ್್ದದೆ್ದದೆ್ು್್ಿದದಿಿೆೆದು್ ್ೆದ ರ         ದೆ್ೆು್್್್ುಿುುಿ  
Evaluation translation (should we go to the mall?) : ('ದುು್್      ುುು್್ುುುಿಿ್್್್್್ಿದುುುುು್ುುುುುುುದುುುುುದದದದದದದು<END>',)
-------------------------------------------
Iterati

Evaluation translation (should we go to the mall?) : ('ಇದರ್ಯ ಸ್ರ್ಯ ಸ್ರ್ಯ ಸ್ರ್ರು.<END>',)
-------------------------------------------
Iteration 2200 : 2.508791208267212
English: But certain things...
Kannada Translation: ಆದರೆ ಕೆಲ .
Kannada Prediction: ಆದರ್ ಇಾಯ್ಇ
Evaluation translation (should we go to the mall?) : ('ಇದರ್ಯ ಆರ್ನ್ನ್ನ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ಲ್ದೆ.<END>',)
-------------------------------------------
Iteration 2300 : 2.652200937271118
English: The locals immediately rushed them to the hospital.
Kannada Translation: ಸ್ಥಳೀಯರು ಕೂಡಲೇ ಅಸ್ವಸ್ಥಗೊಂಡ ಮಹಿಪಾಲನನ್ನು ಆಸ್ಪತ್ರೆಗೆ ದಾಖಲಿಸಿದ್ದಾರೆ.
Kannada Prediction: ಆಿನ್ಿನ   ಮಾರ್್ ಮವಿಯಿ್ಯಿಳಲದೆಹಾಾ ್ಗಿುುಲ್ ಮತಿನಿ್ಲೆ.ೆ.ಅ್ಗೆಿದಿದುಲೆ ು.
Evaluation translation (should we go to the mall?) : ('ಅವ್ನ್ಯ ಅವಾಗಳು ಅವಾಗಿ ಅವಾಗಿದೆ.<END>',)
-------------------------------------------
Iteration 2400 : 2.598966598510742
English: How to make tulsi tea
Kannada Translation: ಚಹಾ ಕುದಿಸುವುದು ಹೇಗೆ
Kannada Prediction: ನಾಿನಮ್ ್ ್   ್ ನಾ
Evaluation translation (should we g

Evaluation translation (should we go to the mall?) : ('ಅವರ್ಯ ಮಾರ್ರ ಮಾರ್ಯ ಮಾರ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ದೆ.<END>',)
-------------------------------------------
Iteration 4300 : 2.6475298404693604
English: PAC meeting
Kannada Translation: ಪಂ ಕಾರ್ಯಪಡೆ ಸಭೆ
Kannada Prediction: ನ್ದಹಾರಿತ ್ಿ ಕ್ಾೕ                                                                                                                                                                                        
Evaluation translation (should we go to the mall?) : ('ಇದರೆ ಮಾರ್ತ್ ಮಾರಿಯ ಮಾರಿಸಿಸ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ತ್ದೆ.<END>',)
-------------------------------------------
Iteration 4400 : 2.7035839557647705
English: Sources at the hospital said that the child and the mother were healthy.
Kannada Translation: ತಾಯಿ ಮಗು ಆರೋಗ್ಯವಾಗಿದ್ದಾರೆ ಎಂದು ಆಸ್ಪತ್ರೆ ಮೂಲಗಳು ತಿಳಿಸಿವೆ.
Kannada Prediction: ಪ್ರಾಯಪಾ್ ಪತ್ದೆ  ುರಿ  ದ    ಸಂದ  ನರ್ರ್್   ಸಾರಿಿು ಪ್ರಿದಿದಾ 
Evaluation translati

Evaluation translation (should we go to the mall?) : ('ಇದರೆ ಕಾರ್ಯಾರ ಕಾರ್ಯಾಗೆ ಕಾಡಿದೆ.<END>',)
-------------------------------------------
Iteration 6200 : 2.641880512237549
English: Frankly, we have little power over what may develop tomorrow, good or bad.
Kannada Translation: ನಾಳೆ ಒಳ್ಳೇದಾಗುವುದೋ ಕೆಟ್ಟದ್ದಾಗುವುದೋ ಇದ್ಯಾವುದರ ಮೇಲೂ ನಮಗೆ ನಿಯಂತ್ರಣವಿಲ್ಲ ಎಂಬುದು ಸತ್ಯ.
Kannada Prediction: ನಮರ್ ಮಂ್ ಿ  ಗಿ ರ ರಗಪ್ಚ್ ್್ರ್ಗಿ ು ್ಗಪಂ್ದೆಗರ ೆೆಮಾಂ್ ಪ್ಾ್ ನ್ಸಾದ್ತಿಿಾದ್ಲ.ಮಂದಿತ್.ಪಿ್ತಿ
Evaluation translation (should we go to the mall?) : ('ಅವರು ಅವರ್ಯ ಮಾರ್ಯಾಗಿದೆ.<END>',)
-------------------------------------------
Iteration 6300 : 2.656193971633911
English: According to a Raj Bhavan communique, the new chief minister has kept with himself Home and other key portfolios held by Jayalalithaa besides retaining Finance and PWD.
Kannada Translation: ರಾಜ ಭವನದ ಮೂಲಗಳ ಪ್ರಕಾರ ನೂತನ ಮುಖ್ಯಮಂತ್ರಿಯವರು ಜಯಾ ಅವರು ನಿರ್ವಹಿಸುತ್ತಿದ್ದ ಗೃಹ, ಹಣಕಾಸು ಹಾಗೂ ಲೋಕೋಪಯೋಗಿ ಮತ್ತಿತರ ಮಹತ್ವದ ಖಾತೆಗಳನ್ನು ತಮ್ಮ ಬಳಿಯೇ ಇಟ್ಟುಕೊಂಡಿದ್ದಾರೆ.
Kannada Prediction: ಆಾವ್

Evaluation translation (should we go to the mall?) : ('ಆದರೆ ಅವರ್ಯಾಗಿ ನಿರು ನಾಡಿದ್ದಾರೆ.<END>',)
-------------------------------------------
Iteration 8400 : 2.268226146697998
English: Meanwhile, police said that the accused tried to flee and were killed during the attempt.
Kannada Translation: ಅಷ್ಟರಲ್ಲಿ, ತಮ್ಮ ಮೇಲೆ ಹಲ್ಲೆ ನಡೆಸಿ ಪರಾರಿಯಾಗಲು ಆರೋಪಿ ಯತ್ನಿಸಿದ್ದ ಎಂದು ಪೊಲೀಸರು ತಿಳಿಸಿದ್ದಾರೆ.
Kannada Prediction: ಆವ್ಟಾ ್ಲಿ  ಮಿ್ಯಾಮಾರ್ ನಾ್ಲಿ ಮಿಿ ್ದಮ್್ರ್ಸ ರಿ್ ಮರ್ಗ್ಸಮಾ್ಯ್ಸಿದೆದೆಮಂದು.ಮ್ಂ್ಸಿ್ವಸ್ರಿದಿದ್ದಾರೆ.
Evaluation translation (should we go to the mall?) : ('ಅವರು ಸಾವು ಸಂದು ಸಿಳಿದ್ದೆ.<END>',)
-------------------------------------------
Iteration 8500 : 2.3383889198303223
English: Your smallest contribution will count a lot for us.
Kannada Translation: ನಿಮ್ಮ ಅಲ್ಪ ಸಹಾಯವು ನಮಗೊಂದು ಮಹತ್ತರವಾದ ವರದಾನ ಆಗುವುದು.
Kannada Prediction: ಆಿಮ್ಮಾಸವ್ಲಿಸಂಾರ ಾ ಸಿಾೆಂದ  ಸಾಾ್ತಿ ಾಗೆಹಿ್ುರ್ನರ್ತುದು.
Evaluation translation (should we go to the mall?) : ('ನಾವು ನಿಮ್ಮಾಗಿ ನಿಮಾಗಿದ್ದಾರೆ.<END>',)
-------------------------------------------
Iter

...

Iteration 12600 : 2.221754312515259
English: Consider three common challenges you might face and how the wisdom found in the Bible can help you.
Kannada Translation: ಮಕ್ಕಳಿಗೆ ಒಂದು ದೊಡ್ಡ ಕಾಯಿಲೆ ಇದೆ ಅಂತ ತಿಳಿದುಬಂದಾಗ ಹೆತ್ತವರ ಕರುಳು ಕಿತ್ತುಬಂದಂತಾಗುತ್ತೆ.
Kannada Prediction: ಈಾ್ಕೆುಗೆ ಸಂದರ ಸೇಂಿಳಿಸಾರಾಸ್ ಮಂುಲಮಂದ್ಆಿಳಿಸ್ ೇದುಗಿಪೊಚ್ತುನಿಮೊ್ತಿವಮೊಂ್ತಿ ್ದುತಿರಿವ್ತಿ 
Evaluation translation (should we go to the mall?) : ('ಆದರೆ ಮಾಡುವುದು ಮಾಡುವುದು?<END>',)
-------------------------------------------
Iteration 12700 : 2.281909704208374
English: Dont skip meals.
Kannada Translation: ಆಹಾರದಲ್ಲಿ ವ್ಯತ್ಯಯ ಮಾಡಿಕೊಳ್ಳಬೇಡಿ.
Kannada Prediction: ಆದರಗ ಲ್ಲಿ ಮಿಯಾಿತವಾಸಾಡಿದೊಳ್ಳುೇಕುದ
Evaluation translation (should we go to the mall?) : ('ಇದರೆ ಅವರ ಸಂದರ ಅವರು ಇಲ್ಲಿ ಇಲ್ಲ.<END>',)
-------------------------------------------
Iteration 12800 : 2.2205915451049805
English: That correct perspective is important if we are to keep our eyes on this big issue and take our stand for Jehovahs righteous rule.
Kannada Translation: ಮುಖ್ಯವಾದ ವಿವಾದಾಂ

Evaluation translation (should we go to the mall?) : ('ಅವರು ಅವರು ಅವರು ನಿರಿಸುತ್ತು?<END>',)
-------------------------------------------
Iteration 14700 : 2.164233446121216
English: President Pranab Mukherjee completes his five-year term on July 24
Kannada Translation: ಜುಲೈ ಇಪ್ಪತ್ನಾಲ್ಕನೇ ತಾರೀಕಿಗೆ ಪ್ರಣವ್ ಮುಖರ್ಜಿ ಅವರ ಅಧಿಕಾರಾವಧಿ ಪೂರ್ಣಗೊಳ್ಳಲಿದೆ
Kannada Prediction: ಮನರೀದಮಲ್ ಿ್ರು   ೆ್ ಕಿರ್ಯ್ ಳ ಪ್ರಿ ಾ ಕತಖ್ಿ ೆ ಮವರುಮಧಿ ್ರ್ಗಣಿ ಮ್ಲ್ಯ ಳಳಿಳು್ ೆ
Evaluation translation (should we go to the mall?) : ('ಅದು ಅವರು ಅವರು ಅವರು ಇಲ್ಲ.<END>',)
-------------------------------------------
Iteration 14800 : 2.1257483959198
English: True success is not determined by the material or social goals that those in the world often pursue.
Kannada Translation: ರಾಜ ಸೊಲೊಮೋನ ಈ ಸತ್ಯವನ್ನು ಅರಿತಿದ್ದನು.
Kannada Prediction: ಇಾಜ್ಪಂಲ್ ್ಗೆಪ ಸಂ್ತಾಾ್ನು ಮವ್ಗ್ ್ದರು 
Evaluation translation (should we go to the mall?) : ('ಇದು ಅವರು ಅವರು ಅವರು ಇದು ಬೇಕು?<END>',)
-------------------------------------------
Iteration 14900 : 2.0032002925872803
Engl

Iteration 16700 : 2.137535810470581
English: He wrote about 80 books.
Kannada Translation: ಸುಮಾರು 80 ಪುಸ್ತಕಗಳನ್ನು ಬರೆದಿರುವ ಹೊ.
Kannada Prediction: ಇಿಮಾನ್ ಕ  ಸ್ಸ್ ಿ್ಳು್ನು ಮಿ್ದಿದು ುಕೇಂ
Evaluation translation (should we go to the mall?) : ('ಅವರು ನಿಮ್ಮ ನಿಮಾನಿಸಿದ್ದಾರೆ?<END>',)
-------------------------------------------
Iteration 16800 : 2.19476056098938
English: 10 lakh.
Kannada Translation: ತಾವು 1.10 ಲಕ್ಷ ರು.
Kannada Prediction: 1ಾಂಿ 10  01ಕ್ಷ 1ೂ.
Evaluation translation (should we go to the mall?) : ('ಅವರ ಕೆಲವು ಕೆಲವಾಗಿ ಯಾರು ಹೇಳಿದೆ?<END>',)
-------------------------------------------
Iteration 16900 : 1.8085999488830566
English: Who were present?
Kannada Translation: ಯಾರ್ಯಾರು ಉಪಸ್ಥಿತರಿದ್ದರು?
Kannada Prediction: ನಾವುಥಾಯಿವಯದ್್ ಿ?ಿಿ?್ದೇೆ?
Evaluation translation (should we go to the mall?) : ('ನಾನು ನಿರ್ಮ ಹೇಗೆ ಯಾರಿಗೆ ಯಾರಿ?<END>',)
-------------------------------------------
Iteration 17000 : 2.1332249641418457
English: How had that happened?
Kannada Translation: ಹೇಗೆ - ಇದು ಹೇಗೆ ಸಂಭವಿಸಿತು?
Kanna

Evaluation translation (should we go to the mall?) : ('ನಾವು ಅವರು ಏನು ಏನು ಏನು ಏನು?<END>',)
-------------------------------------------
Iteration 19000 : 2.1115195751190186
English: The findings showed that taking a nap during the day was associated with an average 5 mm Hg drop in blood pressure.
Kannada Translation: ಆ ದಿನಗಳಲ್ಲಿ ಎನ್ಎಪಿ ತೆಗೆದುಕೊಳ್ಳುವಿಕೆಯು ರಕ್ತದೊತ್ತಡದಲ್ಲಿ ಸರಾಸರಿ 5 ಎಂಎಂ ಎಚ್ಜಿ ಡ್ರಾಪ್ಗೆ ಸಂಬಂಧಿಸಿದೆ ಎಂದು ಸಂಶೋಧನೆಗಳು ತೋರಿಸಿಕೊಟ್ಟವು.
Kannada Prediction: ಇದಕರನ್ೆ ್ಲಿ ಮಂ್ನಂ್ಸಕಿರೆ ು ೊಂ್ಳು  ದೆ   ಕಾ್ಷು ಂ್ತಿ  ್ಲಿ ಸ್್ಧ್್ ಕ ಮಂದಸದಮಂ್ ಿ ಮೆ ಾನ್ ಳ ಮ್ತ್ಧಿ ಿದ್.ಸಂದು ಅ್ದ್ ಿ್ ಳ್ ಮಿರಿಸಿದೊಂ್ಟ್ಾ.
Evaluation translation (should we go to the mall?) : ('ಅವರು ಅವರು ಮಾಡಿದ್ದಾರೆ?<END>',)
-------------------------------------------
Iteration 19100 : 1.86173677444458
English: The children tend to catch infections in the school.
Kannada Translation: ಜಂತುಹುಳ ಬಾಧೆಯಿಂದ ಮಕ್ಕಳು ಹೆಚ್ಚಾಗಿ ರೋಗಗ್ರಸ್ತರಾಗಿ ಶಾಲೆಗೆ ಗೈರಾಗುತ್ತಾರೆ.
Kannada Prediction: ಈನಗರ ಾದ ಮಗರಿ  ತದ ಸಾ್ಕೆ  ಸೆಚ್ಚಿ ಿದಮಾಗುಳಯೆ್ ಿಿ ಿದಮ್ವಿ ಳ ಹೆದುಗಿತ್ತದರೆ.
Evaluation

Iteration 20900 : 2.0068116188049316
English: The poor farmers of the region have got benefitted from this.
Kannada Translation: ಇದರಿಂದಾಗಿ ಈ ಭಾಗದ ಬಡ ರೈತರಿಗೆ ಅನುಕೂಲವಾಗಿದೆ.
Kannada Prediction: ಅದುಲಂದ ಗಿ ಅ ಮಾರೂಲಕಿೆಸಾತ್  ೆ ನವ್ಭ್ಂ್ಾಗಿದೆ.
Evaluation translation (should we go to the mall?) : ('ಅವರು ಅವರು ಹೇಗೆ ಹೇಳಿದ್ದು?<END>',)
-------------------------------------------
Iteration 21000 : 1.6376142501831055
English: Apply it on inflamed and rashy areas.
Kannada Translation: ಮೊಡವೆ ಹಾಗೂ ಮೊಡವೆಯ ಕಲೆಗಳು ಇರುವ ಜಾಗಕ್ಕೆ ಇದನ್ನು ಹಚ್ಚಿಕೊಳ್ಳಿ.
Kannada Prediction: ಇತದೆಾಲಮೊಗೂ ಸತತೆು ನಸಾ್ ಳು ಸದುತುಸನನತ್ಕೆ ಹದೆ್ನು ಮೆ್ಚಿತೆಂ್ಳುಸ
Evaluation translation (should we go to the mall?) : ('ಅದು ಹೆಚ್ಚು ಮಾಡುವುದು?<END>',)
-------------------------------------------
Iteration 21100 : 2.047574281692505
English: Shri Biram, Former Managing Director, State Bank of India & IDBI Bank
Kannada Translation: ಧೋಲಾಕಿಯಾ, ಸ್ಟೇಟ್ ಬ್ಯಾಂಕ್ ಆಫ್ ಇಂಡಿಯಾ ಮತ್ತು ಐಡಿಬಿಐ ಬ್ಯಾಂಕ್ ಮಾಜಿ ವ್ಯವಸ್ಥಾಪಕ ನಿರ್ದೇಶಕ ಶ್ರೀ ಬಿ ್ರೀರಾಮ್ ಸಮಿತಿಯ ಸದಸ್ಯರಾಗಿದ್ದಾರೆ.
Kannada 

In [ ]:
# inference
transformer.eval()
def translate(eng_sentence):
    eng_sentence = (eng_sentence,)
    kn_sentence = ("",)
    for word_counter in range(max_sequence_length):
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
        predictions = transformer(eng_sentence,
                                  kn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_kannada[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
        break
    return kn_sentence[0]

In [ ]:
# batch,batch[sentence_num]

In [ ]:
eng_tokenized, kn_tokenized = [], []
for sentence_num in range(batch_size):
    eng_sentence, kn_sentence = batch[0][sentence_num], batch[1][sentence_num]
    eng_tokenized.append( tokenize(eng_sentence, english_to_index, start_token=False, end_token=False) )
    kn_tokenized.append( tokenize(kn_sentence, kannada_to_index, start_token=True, end_token=True) )
eng_tokenized = torch.stack(eng_tokenized)
kn_tokenized = torch.stack(kn_tokenized)

In [ ]:
eng_tokenized

In [153]:
create_masks(batch[0], batch[1])

encoder_self_attention_mask torch.Size([3, 200, 200]): tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
decoder_self_attention_mask torch.Size([3, 200, 200]): tensor([[ 0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09,
         -1.0000e

(tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          ...,
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          ...,
    

In [163]:
class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token=True, end_token=True):

        def tokenize(sentence, start_token=True, end_token=True):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
            tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())
    
    def forward(self, x, end_token=True): # sentence
        x = self.batch_tokenize(x ,end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x


In [164]:
english_sentences

['Hes a scientist.',
 "'But we speak the truth aur ye sach hai ke Gujarat mein vikas pagal hogaya hai,'' Rahul Gandhi further said in Banaskantha",
 '8 lakh crore have been looted.',
 'I read a lot into this as well.',
 'How did mankind come under Satans rival sovereignty?',
 'And then I became Prime Minister.',
 'What about corruption?',
 '"""The shooting of the film is 90 percent done."',
 'the Special Statute',
 '"Then the king said to Ittai the Gittite, ""Why do you also go with us? Return, and stay with the king. for you are a foreigner, and also an exile. Return to your own place."',
 'What happened at the UN General Assembly?',
 'The meeting was attended by Prime Minister Narendra Modi, Home Minister Amit Shah and Defence Minister Rajnath Singh, among others.',
 'It has been under discussion for a long time.',
 'Buses cannot get there.',
 'Why then this tradition was not thought of?',
 'Kashmiri youth join Indian Army',
 'Basic amenities elude this village',
 'Off-budget borrowi